# Analysing a dataset

This project is about analyzing data on movie review scores. 

The dataset is stored in the *fandango_score_comparison.csv* file. It contains information on how major movie review services rated movies. The data originally came from FiveThirtyEight.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy
import scipy.stats as stats
%matplotlib inline

movies = pd.read_csv('data/fandango_score_comparison.csv')
movies.head(3)


Each row represents a single movie. Each column contains information about how the online moview review services RottenTomatoes, Metacritic, IMDB, and Fandango rated the movie.

To make it easier to compare scores across services, the columns were normalized so their scale and rounding matched the Fandango ratings. Any column with the suffix _norm is the corresponding column changed to a 0-5 scale. 

Let us focus on the Metacritic_norm_round and the Fandango_Stars columns, which will let us see how Fandango and Metacritic differ in terms of review scores.



In [ ]:
movies.hist(column = 'Metacritic_norm_round')

In [ ]:
movies.hist(column = 'Fandango_Stars')

The scores in Metacritic spread over a larger range (from 0.5 up to 4.5) than the scores from Fandango Stars which start from 3.0 to 5.0. Low scores do not exist in Fandango Stars.
To compare the highest frequency score, we can see that in Metacritic, it is 3 which is lower than Fandango's which is 4.5.

## Quantifying these differences with metrics 

Metrics include the mean, median and standard deviation.

In [ ]:
mean_Meta = movies['Metacritic_norm_round'].mean()
mean_Fandango = movies['Fandango_Stars'].mean()
print('Mean of Metacritics scores', mean_Meta)
print('Mean of Fandango scores', mean_Fandango)


In [ ]:
median_Meta = movies['Metacritic_norm_round'].median()
median_Fandango = movies['Fandango_Stars'].median()
print('Median of Metacritic scores', median_Meta)
print('Median of Fandango scores', median_Fandango)

The median metacritic score appears higher than the mean metacritic score because a few very low reviews "drag down" the mean. 

The median fandango score is lower than the mean fandango score because a few very high ratings "drag up" the mean.

In [ ]:
std_Meta = movies['Metacritic_norm_round'].std()
std_Fandango = movies['Fandango_Stars'].std()
print('Standard deviation of Metacritics', std_Meta)
print('Standard deviation of Fandango', std_Fandango)

The standard deviation (for a sample or population) is an indicator of how "near/dense" the data are relative to the mean. A small standard deviation indicates that the data are near the mean, large standard deviation indicates that the data are more dispersed.

Fandango ratings appear clustered between 3 and 5, and have a much narrower standard deviation than Metacritic reviews, which go from 0 to 5. The data are more dispersed in Metacritics.

Fandango ratings in general appear to be higher than metacritic ratings.
These may be due to movie studio influence on Fandango ratings, and the fact that Fandango calculates its ratings in a hidden way.

## Findind which movies tend to be the largest outliers.

In [ ]:
movies.plot.scatter(x='Metacritic_norm_round', y='Fandango_Stars')

Several movies appear to have low ratings in Metacritic and high ratings in Fandango, or vice versa. 
E.g., the movie rated with the lowest score in Metacritic, i.e. 0.5, is rated 3.5 in Fandango...

* Let us explore this further, finding the differences between the columns.

In [ ]:
fm_diff = movies['Metacritic_norm_round'] - movies['Fandango_Stars']
movies['fm_diff'] = numpy.abs(fm_diff)

Getting the absolute value will ensure that we look at any extreme case where Metacritic_norm_round is much greater or much lower than Fandango_Stars.

* Let us sort movies based on the fm_diff column, in descending order

In [ ]:
movies = movies.sort_values(by='fm_diff', ascending=False)

* Let us Print out the top 5 movies with the biggest differences

In [ ]:
movies[['FILM', 'Fandango_Stars', 'Metacritic_norm_round', 'fm_diff']].head(5)

## Measuring the correlation between Fandango_Stars and Metacritic_norm_round 

Let us see what the correlation coefficient between Fandango_Stars and Metacritic_norm_round is. This will help to determine if Fandango consistently has higher scores than Metacritic, or if only a few movies were assigned higher ratings.



In [ ]:
r, p = stats.pearsonr(movies['Fandango_Stars'], movies['Metacritic_norm_round'])
print(r)

The low correlation between Fandango and Metacritic scores indicates that Fandango scores aren't just inflated, they are fundamentally different. For whatever reason, it appears like Fandango both inflates scores overall, and inflates scores differently depending on the movie.

In [ ]:
slope, intercept, r_value, p_value, stderr_slope = stats.linregress(movies["Metacritic_norm_round"], movies["Fandango_Stars"])

## Creating a linear regression

Using the scipy.stats.linregress function, we can create a linear regression with Metacritic_norm_round as the x-values and Fandango_Stars as the y-values.

Let us predict what a movie that got a 3.0 in Metacritic would get on Fandango using the formula pred_3 = slope * 3 + intercept

In [ ]:
pred_3 = slope * 3 + intercept
pred_3

Let us draw the linear regression on our scatter plot:

In [ ]:
pred_1 = 1 * slope + intercept
pred_5 = 5 * slope + intercept
#plt.scatter(movies["Metacritic_norm_round"], movies["Fandango_Stars"])
#plt.plot([1,5],[pred_1,pred_5])
#plt.xlim(1,5)
#plt.show()

ax = movies.plot.scatter(x='Metacritic_norm_round', y='Fandango_Stars')
ax.plot([1,5],[pred_1,pred_5])
ax.set_xlim([1,5])

> Since our correlation coefficient is low, we can see from the graph that predicting Fandango scores from Metacritic scores using linear regression is **not relevant**.

## Plotting using Seaborn: Comparing IMDB scores and Metacritic scores:

In [ ]:
import seaborn as sns
sns.regplot(x='Metacritic_norm_round', y='IMDB_norm_round', data=movies);

In the simplest invocation, both functions draw a scatterplot of two variables, x and y, and then fit the regression model and plot the resulting regression line and a 95% confidence interval for that regression.

A few other seaborn functions use regplot() in the context of a larger, more complex plot including the jointplot() function. In addition to the plot styles, jointplot() can use regplot() to show the linear regression fit on the joint axes by passing kind="reg" as 